In [3]:
# !pip install langchain_community
# !pip install python-dotenv
#! pip install langchain-community

The line _ = load_dotenv(find_dotenv()) is a two-step process. First, find_dotenv() searches for a file named .env by looking in the current directory and then moving up through parent directories until it finds the file. This is useful in projects where the .env file might not be in the same directory as your script. Once the path to the .env file is found, load_dotenv() reads the file and loads any environment variables defined within it into the system environment. This allows sensitive information, like API keys, to be kept out of your source code

In [4]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # Load environment variables from .env file
openai.api_key = os.environ['OPENAI_API_KEY']

Interace with OpenAI's API to get a completion for a given prompt

In the new OpenAI Python library (v1.0.0 and above), a client is an object you create to manage your connection and authentication with the OpenAI API.
The client handles your API key, manages requests, and provides methods (like client.chat.completions.create) to interact with the API.

"user": Represents input from the human interacting with the assistant.
"assistant": Represents responses from the AI model.
"system": (optional) Used to set behavior or context for the assistant.
By specifying "role": "user", you ensure the model treats the message as a prompt it should respond to, rather than as its own previous output or system instructions. This helps the model generate relevant and accurate responses.

Lower temperature (e.g., 0–0.3): More fact-oriented, focused, and deterministic responses. Good for tasks needing accuracy and consistency.
Higher temperature (e.g., 0.7–1): More creative, varied, and open-ended responses. Useful for brainstorming, storytelling, or generating ideas.
  


In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    client = openai.Client()    # create a client instance
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

In [6]:
get_completion("what is 1 + 1?")  # Example usage

'1 + 1 equals 2.'

In [7]:
customer_email = """Betreff: Anfrage

Sehr geehrte Damen und Herren,

ich hoffe, es geht Ihnen gut. Ich möchte mich erkundigen, ob Sie mir bitte weitere Informationen zu Ihrem Angebot zusenden könnten.

Vielen Dank im Voraus für Ihre Unterstützung.

Mit freundlichen Grüßen"""

translate in this style - use this style in you answer

In [8]:
style = """Irish English\n
    in a calm and professional tone" 
    """

In [9]:
prompt = f"""Translate the following email \n
that is delimited by triple backticks into a styke that is {style}.
text: ```{customer_email}```"""
print(prompt)

Translate the following email 

that is delimited by triple backticks into a styke that is Irish English

    in a calm and professional tone" 
    .
text: ```Betreff: Anfrage

Sehr geehrte Damen und Herren,

ich hoffe, es geht Ihnen gut. Ich möchte mich erkundigen, ob Sie mir bitte weitere Informationen zu Ihrem Angebot zusenden könnten.

Vielen Dank im Voraus für Ihre Unterstützung.

Mit freundlichen Grüßen```


In [10]:
rsponse = get_completion(prompt)
print(rsponse)

Subject: Inquiry

Dear Ladies and Gentlemen,

I hope you're keeping well. I'd like to inquire if you could please send me more information about your offer.

Thank you in advance for your assistance.

Kind regards


imagine having to do this with loads of mails/languages. 
lets find a more convienient way to do this with langchain

Chatopenai is langchains abstraction for chatgpt api
use it as a wrapper for openai llm
This class inherits from BaseChatModel and is designed to make it easy to interact with OpenAI's chat APIs, such as GPT-3.5-turbo or GPT-4, by handling configuration, authentication, and API calls in a user-friendly and extensible way.

In [11]:
from langchain_community.chat_models import ChatOpenAI

default temperature is 0.7 so for more accurate response, set it yourself

In [12]:
chat = ChatOpenAI(temperature=0.0)  # Set temperature to 0 for deterministic output

C:\Users\colgann\AppData\Local\Temp\ipykernel_26024\4275512504.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(temperature=0.0)  # Set temperature to 0 for deterministic output


In [11]:
template_string = """
Translate the following email
that is delimited by triple backticks into a style that is {style}.
text: ```{customer_email}```
"""

To repeatedly use this template, import langchain ChatPromptTemplate

Notice how we didnt use the formatted string. Prompt templates are better for more complex, longer prompts
When you print the prompt template u can see it picks up the variables 

langchain provides prompty for common operations like smmarisation or question-answering, connecting to sql dbs, connecting to apis

In [12]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [13]:
print(prompt_template.messages[0].prompt)
print(prompt_template.messages[0].prompt.input_variables)

input_variables=['customer_email', 'style'] input_types={} partial_variables={} template='\nTranslate the following email\nthat is delimited by triple backticks into a style that is {style}.\ntext: ```{customer_email}```\n'
['customer_email', 'style']


In [14]:
customer_style = """Irish English
    in a calm and professional tone" 
    """

In [15]:
customer_email = """Betreff: Anfrage
Sehr geehrte Damen und Herren,
ich hoffe, es geht Ihnen gut. Ich möchte mich erkundigen, ob Sie mir bitte weitere Informationen zu Ihrem Angebot zusenden könnten.
Vielen Dank im Voraus für Ihre Unterstützung.

Mit freundlichen Grüßen"""

Generate prompt

In [16]:
customer_messages = prompt_template.format_messages(
    style=customer_style,
    customer_email=customer_email
)

In [17]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [18]:
print(customer_messages[0])

content='\nTranslate the following email\nthat is delimited by triple backticks into a style that is Irish English\n    in a calm and professional tone" \n    .\ntext: ```Betreff: Anfrage\nSehr geehrte Damen und Herren,\nich hoffe, es geht Ihnen gut. Ich möchte mich erkundigen, ob Sie mir bitte weitere Informationen zu Ihrem Angebot zusenden könnten.\nVielen Dank im Voraus für Ihre Unterstützung.\n\nMit freundlichen Grüßen```\n' additional_kwargs={} response_metadata={}


Pass this prompt to the llm

In [19]:
customer_rsponse = chat(customer_messages)

C:\Users\colgann\AppData\Local\Temp\ipykernel_17472\3043644737.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  customer_rsponse = chat(customer_messages)


In [20]:
print(customer_rsponse.content)

Subject: Inquiry

Dear Sir/Madam,

I hope you are keeping well. I would like to inquire if you could please send me further information about your offer.

Thank you in advance for your assistance.

Kind regards


We can now use this prompt template again & again with different params (style & email)

In [21]:
service_reply = """Hey there, I hope you're doing well. Thank you for reaching out to us. We appreciate your interest in our services. I will be happy to provide you with more information about our offerings. Please give me a moment to gather the details, and I will get back to you shortly. Thank you for your patience!"""

In [22]:
service_style_jamaican = "Jamaican English"

In [23]:
customer_messages_2 = prompt_template.format_messages(
    style=service_style_jamaican,
    customer_email=service_reply
)
print(customer_messages_2)

[HumanMessage(content="\nTranslate the following email\nthat is delimited by triple backticks into a style that is Jamaican English.\ntext: ```Hey there, I hope you're doing well. Thank you for reaching out to us. We appreciate your interest in our services. I will be happy to provide you with more information about our offerings. Please give me a moment to gather the details, and I will get back to you shortly. Thank you for your patience!```\n", additional_kwargs={}, response_metadata={})]


In [24]:
customer_rsponse_2 = chat(customer_messages_2)
print(customer_rsponse_2.content)

Hey deh, mi hope yuh good. Big up fi linkin' wi. Wi appreciate yuh interest inna wi services. Mi gwine happy fi give yuh more info 'bout weh wi have fi offer. Jus' gi mi a likkle time fi gather di details, an' mi gwine get back to yuh soon. Tank yuh fi yuh patience!


# Parsing
How to extract info & output json

In [31]:
#Python dictionary
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "affordable"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'affordable'}

In [32]:
customer_review = """\
This leaf blower is pretty amazing. It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

In [33]:
review_template = """\
for the folwing text, etract the following information:

gift: if the item was purchased as a gif for someone else? ans
delivery_days: how many days did it take to deliver the item
price_value: extract any sentences aboutthe price or value

format the output as json with the following keys:

gift
delivery_days
price_value

text: {text}
"""

In [34]:
prompt_template2 = ChatPromptTemplate.from_template(review_template)
messages = prompt_template2.format_messages(text=customer_review)
chat=ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
  "gift": "yes",
  "delivery_days": 2,
  "price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}


Use Langchain parser to extract json

In [35]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

Tell it what you want it to parse with schemas

In [37]:
gift_schema = ResponseSchema(name="gift", description="was this item a gift? Answer 'True' if it was, 'False' if it was not, and 'unknown' if you dont know")
delivery_days_schema = ResponseSchema(name="delivery_days", description="how many days did it take to deliver the item. Answer 1 for one day, 2 for two days etc., or unknown if you dont know")
price_value_schema = ResponseSchema(name="price_value", description="extract any info aout the price value then output them as csv values")

responseSchemas = [gift_schema, delivery_days_schema, price_value_schema]

Langchain can give you the format instructions to format the output eg. as JSON

You can then include that in your template

In [38]:
output_parser = StructuredOutputParser(response_schemas=responseSchemas)
format_instructions = output_parser.get_format_instructions()

In [39]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // was this item a gift? Answer 'True' if it was, 'False' if it was not, and 'unknown' if you dont know
	"delivery_days": string  // how many days did it take to deliver the item. Answer 1 for one day, 2 for two days etc., or unknown if you dont know
	"price_value": string  // extract any info aout the price value then output them as csv values
}
```


In [40]:
review_template2 = """\
for the folwing text, etract the following information:

gift: if the item was purchased as a gif for someone else? ans
delivery_days: how many days did it take to deliver the item
price_value: extract any sentences aboutthe price or value

text: {text}

{format_instructions}
"""

In [41]:
prompt_template = ChatPromptTemplate.from_template(review_template2)
messages = prompt_template.format_messages(
    text=customer_review,
    format_instructions=format_instructions
    )
chat=ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [ ]:
16:00 models,pr...